In [1]:
# -*- coding: utf-8 -*-
"""
Original Created on Thu 09 Jun 11:56:21 MDT 2022
@author: Daphne Zakarian

Read in the WDS using pandas, turn into an astropy table, find secondary coords, 
account for proper motion etc.

Final table will be used to query Gaia -- most importantly, we have primary 
and secondary stars' coordinates in deg

Changes began on Thu 30 May 09:08:00 MDT 2024 
by Bob Zavala to include Discoverer and Components in order to 
pass unique identifier information along. Coordinates and WDS ID alone
do not uniquely identify the WDS entries. 

Name of file changed to explicitly state the code builds and writes a WDS 
table. 

"""



import pandas as pd
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import ICRS, SkyCoord, FK5
from astropy.io import ascii
import math
import numpy as np
import os

# to time task completion only
import time

this_path = os.getcwd()

    

In [2]:
# Read in Table: fixed width file to pandas to astropy 

# manually list WDS txt file column widths to read in 
# the formatted data. A change made by Daphane is to 
# separate the DM column into two parts and the WDS 
# 2000 arcsecond coordinates into RA and DEC portions. 
## IMPORTANT: colspecs are HALF-OPEN intervals: [,)
## SO THE RIGHT SIDE IS NOT INCLUSIVE!!!
columns = ((0,10),(10,17),(17,23),(23,28),(28,32),(33,37),
           (37,42),(42,45),(45,51),(51,57),(57,64),(64,68),
           (69,80),(80,84),(84,88),(88,93),(93,98),(98,101),
           (101,106),(107,111),(112,121),(121,130))

oldnames= ['WDS_Identifier', 'Discovr', 'Comp', 'Epoch-1', 'Epoch-2', '#', 
           'Theta-1', 'Theta-2', 'Rho-1', 'Rho-2', 'Mag-pri', 'Mag-sec',
           'SpecType','PMpri-RA', 'PMpri-DEC', 'PMsec-RA', 'PMsec-DEC', 'DM', 
           'Desig', 'Note', 'WDS_RA"', 'WDS_DEC"']

# read fixed width file into pandas 
    # easier to work with this fixed-width-formatted (fwf) file 
    # in pandas than astropy
wdspd = pd.read_fwf(this_path+"/wdsweb_summ2.txt",
                    colspecs=columns,header=None,names=oldnames,skiprows=3)

# pandas table -> astropy table
wdstab = Table.from_pandas(wdspd)



In [ ]:
wdstab

In [ ]:
wdstab[2]

In [ ]:
wdstab[2].colnames

In [ ]:
wdstab[2]['Theta-1']

In [ ]:
wdstab[156613]['WDS_RA"']

In [3]:
# After the above quality checks that verify the WDS was read-in 
# correctly REMOVE UNNECESSARY COLUMNS
wdstab.remove_columns(['#','SpecType','DM', 'Desig', 'Note',])

In [5]:
# Establish format for RA and DEC

# make new column to store updated RA and DEC -- is there a better way to
    # make column dtype longer so I don't have to manually type 15 spaces???
wdstab['RAprideg']=0.0
wdstab['DECprideg']=0.0
wdstab['RAhms']='               '
wdstab['DECdms']='               '
 

# assign units to the new columns for coordinates in degrees
wdstab['RAprideg'].unit=u.deg
wdstab['DECprideg'].unit=u.deg


# loop through the ra and dec of each row and make new columns for coords in 2 formats:
    # hms/dms format and degrees
for row in wdstab:
    try:
        ra1 = row['WDS_RA"']
        dec1 = row['WDS_DEC"']
        rastr = ra1[:2] + "h" + ra1[2:4]+"m"+ra1[4:]+"s"
        decstr = dec1[:3] + "d" + dec1[3:5]+"m"+dec1[5:]+'s'
        
        
        # put the strings of ra and dec into the table
        row['RAhms']= rastr
        row['DECdms']= decstr
        
        # put the coordinates into degree form - should work with GAIA
        coo = ICRS(rastr,decstr)
        # put the new coordinates into a column with a float dtype
        row['RAprideg']= coo.ra.deg
        row['DECprideg']=coo.dec.deg
    # skip rows that don't have coordinates
    except ValueError:
        pass

    

""" FIND COORDINATE OF SECONDARY  """

# make new columns for secondary coords
wdstab['RAsecdeg']=0.0
wdstab['DECsecdeg']=0.0
wdstab['RAsecdeg'].unit=u.deg
wdstab['DECsecdeg'].unit=u.deg

startTimeFor = time.time()
c=0
for row in wdstab:
    prira, pridec = row['RAprideg'], row['DECprideg']
    angle, sep = row['Theta-2']*u.deg, row['Rho-2']*u.arcsec
    pricoord = SkyCoord(prira*u.deg, pridec*u.deg, frame='icrs')
    seccoord= pricoord.directional_offset_by(angle, sep)
    row['RAsecdeg']= seccoord.ra.deg
    row['DECsecdeg'] = seccoord.dec.deg
    c+=1
    if c % 1000==0:
        print(c)

endTimeFor = time.time()
elapLoopTime = endTimeFor - startTimeFor
print('Elapsed time for seccoord using for loop method: ',elapLoopTime,' seconds.')





1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
Elapsed time for

In [11]:
pricoord,seccoord.ra

(<SkyCoord (ICRS): (ra, dec) in deg
     (359.97333333, -31.20033333)>,
 <Longitude 359.97312256 deg>)

In [19]:
# Use a vector method to avoid the FOR loop to compute the secondary coordinates

startTimeVec = time.time()
pricoord=SkyCoord(wdstab[0:]['RAprideg'],wdstab[0:]['DECprideg'], frame='icrs')
angle = wdstab[0:]['Theta-2']*u.deg
sep = wdstab[0:]['Rho-2']*u.arcsec
seccoord = pricoord.directional_offset_by(angle, sep)
wdstab[0:]['RAsecdeg'] = seccoord[0:].ra
wdstab[0:]['DECsecdeg'] = seccoord[0:].dec
endTimeVec = time.time()
elapVecTime = endTimeVec - startTimeVec

print('Elapsed time for seccoord using vector method: ',elapVecTime,' seconds.')

len(pricoord)

Elapsed time for seccoord using vector method:  0.06765103340148926  seconds.


156614

In [18]:
pricoord[156613],seccoord[156613].dec

(<SkyCoord (ICRS): (ra, dec) in deg
     (359.97333333, -31.20033333)>,
 <Latitude -31.20040617 deg>)

In [15]:
wdstab[0:]['RAsecdeg'] = seccoord[0:].ra

In [17]:
wdstab[2]

WDS_Identifier,Discovr,Comp,Epoch-1,Epoch-2,Theta-1,Theta-2,Rho-1,Rho-2,Mag-pri,Mag-sec,PMpri-RA,PMpri-DEC,PMsec-RA,PMsec-DEC,"WDS_RA""","WDS_DEC""",RAprideg,DECprideg,RAhms,DECdms,RAsecdeg,DECsecdeg
,,,,,,,,,,,,,,,,,deg,deg,,,deg,deg
str10,str7,str5,int64,int64,int64,int64,float64,float64,str5,str4,float64,float64,float64,float64,str9,str9,float64,float64,str15,str15,float64,float64
00000+4004,ES 2543,AC,1931,2015,69,66,20.0,14.4,12.1,14.1,8.0,-4.0,13.0,8.0,000003.66,+400519.4,0.01525,40.08872222222222,00h00m03.66s,+40d05m19.4s,0.020026521670582886,40.09034907070518


In [ ]:
""" ACCOUNT FOR PROPER MOTION """
# Change units of pm to deg/year
wdstab['PMpri-RAdeg'] = wdstab['PMpri-RA'].to(u.deg/u.year)
wdstab['PMpri-DECdeg'] = wdstab['PMpri-DEC'].to(u.deg/u.year)
wdstab['PMsec-RAdeg'] = wdstab['PMsec-RA'].to(u.deg/u.year)
wdstab['PMsec-DECdeg'] = wdstab['PMsec-DEC'].to(u.deg/u.year)

# Make new columns for the final, Gaia-ready coordinates
wdstab['RApri-prepped']=0.0
wdstab['DECpri-prepped']=0.0
wdstab['RAsec-prepped']=0.0
wdstab['DECsec-prepped']=0.0

# assign unit (degrees) to the new columns
wdstab['RApri-prepped'].unit=u.deg
wdstab['DECpri-prepped'].unit=u.deg
wdstab['RAsec-prepped'].unit=u.deg
wdstab['DECsec-prepped'].unit=u.deg

# use the proper motions to precess coordinates to J2016 if pm info is available in Gaia
# PRIMARY

c=0   
for row in wdstab:
    # bring in all proper motions [deg] and coordinates [deg]
    pmpriRA, pmpriDEC = row['PMpri-RAdeg'], row['PMpri-DECdeg'],

    c+=1
    if c%1000 == 0:
        print(c)
            
    # if there isn't a pm, just use J200 coords
    # this is done for ra and dec separately because a few coords only have pm 
    # in one 'direction'
    if str(pmpriRA) == 'nan':
        row['RApri-prepped'] = row['RAprideg'] 
        if str(pmpriDEC) == 'nan':
            row['DECpri-prepped'] = row['DECprideg']
    elif str(pmpriDEC) == 'nan':
        row['DECpri-prepped'] = row['DECprideg']

    # otherwise, update for pm using SkyCoord
    else:
        coord = SkyCoord(row['RAprideg'], row['DECprideg'], unit='deg', 
                         pm_ra_cosdec = pmpriRA *u.deg/u.year, pm_dec = pmpriDEC *u.deg/u.year)
        newcoord = coord.apply_space_motion(dt=16*u.yr)
        
        row['RApri-prepped'], row['DECpri-prepped'] = newcoord.ra.deg, newcoord.dec.deg


# use the proper motions to precess coordinates to J2016 if pm info is available in Gaia
# SECONDARY       
c=0   
     
for row in wdstab:
    # bring in all proper motions [deg] and coordinates [deg]
    pmsecRA, pmsecDEC = row['PMsec-RAdeg'], row['PMsec-DECdeg']
    
    c+=1
    if c%1000 == 0:
        print(c)
    
    # if there isn't a pm, just use 2016 coords
    if str(pmsecRA) == 'nan':
        row['RAsec-prepped'] = row['RAsecdeg'] 
        if str(pmsecDEC) == 'nan':
            row['DECsec-prepped'] = row['DECsecdeg']
    elif str(pmsecDEC) == 'nan':
        row['DECsec-prepped'] = row['DECsecdeg']
        
    # otherwise, update for pm
    else:
        coord = SkyCoord(row['RAsecdeg'], row['DECsecdeg'], unit='deg', 
                         pm_ra_cosdec = pmsecRA *u.deg/u.year, pm_dec = pmsecDEC *u.deg/u.year)
        newcoord = coord.apply_space_motion(dt=16*u.yr)
        
        row['RAsec-prepped'], row['DECsec-prepped'] = newcoord.ra.deg, newcoord.dec.deg

""" SAVE TABLE AS ECSV and CSV """

ascii.write(wdstab, this_path+'/wdstab_new.ecsv', format='ecsv')
ascii.write(wdstab, this_path+'/wdstab_new.csv', format='csv')



